# Imports

In [ ]:
import numpy as np

import longitudinal_tomography.tomography.tomography as tmo
import longitudinal_tomography.tracking.particles as parts

import longitudinal_tomography.utils.tomo_input as tomoin
import longitudinal_tomography.utils.tomo_output as tomoout
from longitudinal_tomography.tracking import tracking

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Preparation

In [ ]:
with open('./data/c598_1.0turns.dat', 'r') as file:
    raw_params, raw_data = tomoin._split_input(file.readlines())

In [ ]:
machine, frames = tomoin.txt_input_to_machine(raw_params)
machine.values_at_turns()
measured_waterfall = frames.to_waterfall(raw_data)

In [ ]:
plt.imshow(measured_waterfall)
plt.gca().set_aspect(3)

plt.xlabel('Profile bin')
plt.ylabel('#Profile')
plt.colorbar(label='Density');

In [ ]:
profiles = tomoin.raw_data_to_profiles(
    measured_waterfall, machine, frames.rebin, frames.sampling_time)
profiles.calc_profilecharge()

In [ ]:
if profiles.machine.synch_part_x < 0:
    fit_info = dtreat.fit_synch_part_x(profiles)
    machine.load_fitted_synch_part_x_ftn(fit_info)


In [ ]:
reconstr_idx = machine.filmstart

In [ ]:
tracker = tracking.Tracking(machine)

In [ ]:
xp, yp = tracker.track(reconstr_idx)

In [ ]:
# Converting from physical coordinates ([rad], [eV])
# to phase space coordinates.
if not tracker.self_field_flag:
    xp, yp = parts.physical_to_coords(
        xp, yp, machine, tracker.particles.xorigin,
        tracker.particles.dEbin)


In [ ]:
xp, yp = parts.ready_for_tomography(xp, yp, machine.nbins)

In [ ]:
waterfall = profiles.waterfall

# Tomographic Reconstruction

In [ ]:
# Reconstructing phase space
tomo = tmo.TomographyCpp(waterfall, xp, )
weight = tomo.run(niter=machine.niter)

In [ ]:
phasespace = tomoout.create_phase_space_image(
    xp, yp, weight, machine.nbins, reconstr_idx)

In [ ]:
tomoout.show(phasespace, tomo.diff, waterfall[reconstr_idx])